### This script runs the HITL-ABC on the data-sets from Turin model. 
The data-sets are generated using Turin_generateData.R and TurinMisspecification.R

In [1]:
import pyreadr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.stats import gaussian_kde
from scipy import stats
import functions as f
import random

In [19]:
# Loading data-sets
data_param = np.asarray(pyreadr.read_r('Turin_param_miss')[None])
data_param = data_param[:,:3] # Parameters

# Observed statistics after corruption
# s_obs = np.asarray(pyreadr.read_r('Turin_sobs_zeta1')[None]).T # zeta = 1
s_obs = np.asarray(pyreadr.read_r('Turin_sobs_zeta5')[None]).T # zeta = 5
# s_obs = np.asarray(pyreadr.read_r('Turin_sobs_zeta10')[None]).T # zeta = 10

data_sim = np.asarray(pyreadr.read_r('Turin_ssim_miss')[None])
s_sim = data_sim[:,:6] # Simulated data

In [21]:
# Settings
G0_start = 1e-10
G0_end = 3e-9
T_start = 1e-9
T_end = 2e-8
lambda_start = 1e8
lambda_end = 4e9

parameters = {
    'G0':{
        'display_name': '$G_0$',
        'val': 1e-9
        },
    'T':{
        'display_name': 'T',
        'val': 1e-8
        },
    'lambda':{
        'display_name': '$\lambda$',
        'val': 1e9
        }
    }

# Uniform prior specifications
priors = {
    'G0':{
        'start': G0_start,
        'stop': G0_end
        },
    'T':{
        'start': T_start,
        'stop': T_end
        },
    'lambda':{
        'start': lambda_start,
        'stop': lambda_end
        }
    }

In [33]:
M = 2000 # No. of samples from prior
M_epsilon = 0.05 # Tolerance threshold
nGSamples = 4000 # No. of samples used to estimate the KL divergence
nStats = 6 # No. of statistics

param = data_param[0:M, :]
param = (param - np.mean(param, 0)) / np.std(param, axis = 0) # Standardizing the parameters for KL divergence computation

# parameters
rho = 0.5
pi = 0.95
delta = 0.06
nParam = len(parameters)

gamma_true = np.ones(nStats)
gamma_true = [1,1,1,0,1,1]

# Computing prior predictive probability of f
predProb_f1 = rho * pi + (1 - rho) * (1 - pi)
predProb_f0 = 1 - predProb_f1

queried_idx = []  # Initializing an empty list that will contain the indices of queried statistics
feedbacks = []  # Initializing an empy list that will contain the feedbacks

k = 0
d = 1
while (k < nStats and d > delta):
    # Initializing vector of utilities
    utility = np.zeros(nStats)

    gamma_vector_current = f.sample_gamma(queried_idx, feedbacks, nGSamples, nStats, pi, rho)
    samples_current = f.sample_ABC_Posterior(s_obs, param, s_sim, M_epsilon, gamma_vector_current)

    for j in range(nStats):
        if j not in queried_idx:  # Skipping the utility computation if the jth statistic has already been queried
            # Sample from p_ABC(theta | y, F, fj = 1)
            gamma_vector_new1 = f.sample_gamma(queried_idx + [j], feedbacks + [1], nGSamples, nStats, pi, rho)
            samples_future_f1 = f.sample_ABC_Posterior(s_obs, param, s_sim, M_epsilon, gamma_vector_new1)
            
            # Sample from p_ABC(theta | y, F, fj = 0)
            gamma_vector_new0 = f.sample_gamma(queried_idx + [j], feedbacks + [0], nGSamples, nStats, pi, rho)
            samples_future_f0 = f.sample_ABC_Posterior(s_obs, param, s_sim, M_epsilon, gamma_vector_new0)
            # Computing KL divergence
            KL_estimate_f1 = f.KLdivergence(samples_future_f1, samples_current)
            KL_estimate_f0 = f.KLdivergence(samples_future_f0, samples_current)
            # Computing utility
            utility[j] = predProb_f1 * KL_estimate_f1 + predProb_f0 * KL_estimate_f0


    print('Iteration no.' + str(k+1) + ', utilities:')
    print(utility)
    d = utility.max()
    if d > delta:
        print('The queried statistic is no.' + str(np.argmax(utility)+1))
        queried_idx.append(np.argmax(utility))
        feedbacks.append(gamma_true[np.argmax(utility)])
        print('Obtained feedback: ' + str(feedbacks[-1]) + '\n')
    else:
        print('We do not need to query another feedback from the expert! \n')
    k = k + 1

gamma_hat = np.zeros(nStats)
gamma_hat[queried_idx] = feedbacks
print('Obtained gamma_hat: ' + str(gamma_hat))
print('Generating final ABC results...')
samples_final = f.regression_ABC(s_obs, param, s_sim, M_epsilon, gamma_hat.astype('bool'))


Iteration no.1, utilities:
[0.34091135 0.28516855 0.26245663 0.46706563 0.3205958  0.27314045]
The queried statistic is no.4
Obtained feedback: 0

Iteration no.2, utilities:
[0.22031764 0.19786801 0.14488314 0.         0.14562931 0.1201123 ]
The queried statistic is no.1
Obtained feedback: 1

Iteration no.3, utilities:
[0.         0.2811417  0.17986837 0.         0.10181491 0.16829819]
The queried statistic is no.2
Obtained feedback: 1

Iteration no.4, utilities:
[0.         0.         0.11936822 0.         0.18377852 0.13063004]
The queried statistic is no.5
Obtained feedback: 1

Iteration no.5, utilities:
[0.         0.         0.10388221 0.         0.         0.03817821]
The queried statistic is no.3
Obtained feedback: 1

Iteration no.6, utilities:
[0.         0.         0.         0.         0.         0.06236539]
The queried statistic is no.6
Obtained feedback: 1

Obtained gamma_hat: [1. 1. 1. 0. 1. 1.]
Generating final ABC results...
